## 前準備

PostgreSQLを立ち上げ (Mac)

```bash
brew services start postgresql@14                                                                                                
```

PostgreSQLのデータベースを作成しておく

In [1]:
!createdb ml-workshop

## .envに保存された設定ファイルを読み込み

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt")

docs = loader.load_and_split()

In [4]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

## pgvectorを試す
### PostgresSQLにembeddingを保存

In [5]:
from langchain_community.vectorstores.pgvector import PGVector

pgvector_db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name="state_of_the_union",
    connection_string="postgresql+psycopg2://ken@localhost:5432/ml-workshop",
)

/Users/ken/.pyenv/versions/3.11.5/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:289: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


### LangChainにより自動的に作成されているテーブルの確認

In [6]:
!psql -d ml-workshop -c "\dt"

                List of relations
 Schema |          Name           | Type  | Owner 
--------+-------------------------+-------+-------
 public | langchain_pg_collection | table | ken
 public | langchain_pg_embedding  | table | ken
(2 rows)



In [7]:
!psql -d ml-workshop -c "\d langchain_pg_embedding"

               Table "public.langchain_pg_embedding"
    Column     |       Type        | Collation | Nullable | Default 
---------------+-------------------+-----------+----------+---------
 collection_id | uuid              |           |          | 
 embedding     | vector            |           |          | 
 document      | character varying |           |          | 
 cmetadata     | json              |           |          | 
 custom_id     | character varying |           |          | 
 uuid          | uuid              |           | not null | 
Indexes:
    "langchain_pg_embedding_pkey" PRIMARY KEY, btree (uuid)
Foreign-key constraints:
    "langchain_pg_embedding_collection_id_fkey" FOREIGN KEY (collection_id) REFERENCES langchain_pg_collection(uuid) ON DELETE CASCADE



### pgvectorで検索

In [8]:
pgvector_db.similarity_search("test")

[Document(page_content='One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. \n\nWhen they came home, many of the world’s fittest and best trained warriors were never the same. \n\nHeadaches. Numbness. Dizziness. \n\nA cancer that would put them in a flag-draped coffin. \n\nI know. \n\nOne of those soldiers was my son Major Beau Biden. \n\nWe don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. \n\nBut I’m committed to finding out everything we can. \n\nCommitted to military families like Danielle Robinson from Ohio. \n\nThe widow of Sergeant First Class Heath Robinson.  \n\nHe was born a soldier. Army National Guard. Combat medic in Kosovo and Iraq. \n\nStationed near Baghdad, just yards from burn pits the size of football fields. \n\nHeath’s widow Danielle is here with us tonight. They loved going to Ohio State football games. H

### 作成したDBを削除

In [9]:
!dropdb -f ml-workshop

## Chroma DBを試す
### Chromaにembeddingを保存

In [10]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(
    docs,
    embeddings,
    collection_name="state_of_union"
)

### Chromaで検索

In [11]:
chroma_db.similarity_search("test")

[Document(page_content='One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. \n\nWhen they came home, many of the world’s fittest and best trained warriors were never the same. \n\nHeadaches. Numbness. Dizziness. \n\nA cancer that would put them in a flag-draped coffin. \n\nI know. \n\nOne of those soldiers was my son Major Beau Biden. \n\nWe don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. \n\nBut I’m committed to finding out everything we can. \n\nCommitted to military families like Danielle Robinson from Ohio. \n\nThe widow of Sergeant First Class Heath Robinson.  \n\nHe was born a soldier. Army National Guard. Combat medic in Kosovo and Iraq. \n\nStationed near Baghdad, just yards from burn pits the size of football fields. \n\nHeath’s widow Danielle is here with us tonight. They loved going to Ohio State football games. H